In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy import fftpack
from scipy.integrate import simps
from ipywidgets import interact

In [2]:
df = []

start_num = 4
end_num = 31
for num in range(start_num, end_num + 1):
    file_path = f"C:/charpy/230724_{num:04d}.csv"
    df.append(pd.read_csv(file_path, skiprows=45, encoding='shift-jis'))

start_num = 1
end_num = 40
for num in range(start_num, end_num + 1):
    file_path = f"C:/charpy/231005_{num:04d}.csv"
    df.append(pd.read_csv(file_path, skiprows=45, encoding='shift-jis'))

start_num = 1
end_num = 68

#c:/Users/syuuy/OneDrive/charpy/absorb_energy.xlsx
#5列目のデータをabsorb_energyに取り出す
absorb_energy = pd.read_excel("C:/charpy/absorb_energy.xlsx", usecols=[4])

#absorb_energyを1次元に変換
absorb_energy = absorb_energy.values.flatten()

#列名がdensity,impact bendingのデータフレームを作成
charpy_df = pd.read_excel('C:/charpy/absorb_energy.xlsx', usecols=[5,6])
charpy_df.columns = ['density', 'impact bending']

In [3]:
#x[i]にdf[i].iloc[:, 0]を代入
x1 = [0] * (end_num - start_num + 1)
y1 = [0] * (end_num - start_num + 1)
x = [0] * (end_num - start_num + 1)
y = [0] * (end_num - start_num + 1)
for i in range(end_num - start_num + 1):
    x1[i] = df[i].iloc[:, 0]
    y1[i] = df[i].iloc[:, 1]
    #ya[i]にy[i]>0.01の条件で切り取る

x = x1
y = y1

'''
for i in range(end_num - start_num + 1):
    for j in range(len(x1[i])):
        if x1[i][j] > 0.013 and 0.02 > x1[i][j]:
            x[i] = np.append(x[i], x1[i][j])
            y[i] = np.append(y[i], y1[i][j])
    x[i] = np.delete(x[i], 0)
    y[i] = np.delete(y[i], 0)
'''


'\nfor i in range(end_num - start_num + 1):\n    for j in range(len(x1[i])):\n        if x1[i][j] > 0.013 and 0.02 > x1[i][j]:\n            x[i] = np.append(x[i], x1[i][j])\n            y[i] = np.append(y[i], y1[i][j])\n    x[i] = np.delete(x[i], 0)\n    y[i] = np.delete(y[i], 0)\n'

In [4]:
m = 13.742
v0 = 3.779894129

#荷重P=ma
def calc_P(a):
    a = np.array(a)
    return m*a
        
#速度v = v0 - (a * dt)を0からtまで積分
def calc_v(a, secs):
    vt = np.zeros((end_num - start_num + 1, len(secs[0])),dtype=float)
    for i in range(end_num - start_num + 1):
        for j in range(len(secs[i])):
            vt[i][j] = v0 - simps(a[i][0:j+1], x[i][0:j+1])
    return vt
        
#たわみd = vを0からtまで積分
def calc_d(velocity,secs):
    d = np.zeros((end_num - start_num + 1, len(secs[0])),dtype=float)
    for i in range(end_num - start_num + 1):
        for j in range(len(secs[i])):
            d[i][j] = simps(velocity[i][0:j+1], x[i][0:j+1])
    return d

#衝撃曲げ仕事 e = 0.5 * m * (v0**2 - v**2)を計算
def calc_e(velocity):
    et = np.zeros((end_num - start_num + 1, len(x[0])),dtype=float)
    for i in range(end_num - start_num + 1):
        for j in range(len(x[i])):
            et[i][j] = 0.5 * m * (v0**2 - velocity[i][j]**2)
    return et

#eの最大値を取り出す
def calc_e_max(energy):
    energy_max = np.zeros(end_num - start_num + 1,dtype=float)
    for i in range(end_num - start_num + 1):
        energy_max[i] = np.max(energy[i])
    energy_max = np.array(energy_max)
    return energy_max

#absorb_energyをe_maxで引く
def calc_e_diff(energy_max):
    energy_diff = absorb_energy - energy_max
    return energy_diff

#absorb_energyとe_maxのMAE, RMSE, R2を計算
def calc_error(energy_max):
    energy_diff = calc_e_diff(energy_max)
    mean_absolute_error = np.mean(np.abs(energy_diff))
    root_mean_squared_error = np.sqrt(np.mean(energy_diff**2))
    coefficient_determination = 1 - np.sum(energy_diff**2) / np.sum((absorb_energy - np.mean(absorb_energy))**2)
    return mean_absolute_error, root_mean_squared_error, coefficient_determination

In [20]:
def savitzky(y, window_length, polyorder):
    y = np.array(y)
    return signal.savgol_filter(y, window_length, polyorder)

def filter(num, window_length1, polyorder1, window_length2, polyorder2):
    y_sg = savitzky(y, window_length1, polyorder1)
    y_sg_sg = savitzky(y_sg, window_length2, polyorder2)
    plt.figure(figsize=(10, 6))
    plt.title('Savitzky-Golay filter')
    plt.xlabel('time')
    plt.ylabel('acceleration')
    plt.legend(loc='best')
    plt.xlim(0.013,0.02)
    plt.grid()
    plt.plot(x[num], y[num],label='Savitzky-Golay',alpha=0.5)
    plt.plot(x[num], y_sg_sg[num],label='SG',c='red')
    plt.show()

interact(filter, 
        num=(0, 67, 1), 
        window_length1=(9, 31, 2), 
        polyorder1=(1, 5, 1),
        window_length2=(9, 31, 2),
        polyorder2=(1, 5, 1))

interactive(children=(IntSlider(value=33, description='num', max=67), IntSlider(value=19, description='window_…

<function __main__.filter(num, window_length1, polyorder1, window_length2, polyorder2)>